# Callable Bonds


## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2025

***

# Pricing a Callable Bond

## Freddie Mac

The U.S. has many forms of *agency* debt in addition to *Treasury* debt. The largest of these *agency* issuers are the housing entities commonly referred to as Freddie Mac and Fannie Mae. While technically distinct from the U.S. Treasury, they are widely seen as having the full credit and backing of the Treasury--particularly after the bailouts of 2008.

Thus, we will examine this agency debt as (credit) risk-free, just like Treasury debt.
* Freddie Mac typically trades about 20bps higher yield than treasuries. 
* Though this may reflect some credit risk, it may reflect mostly an illiquidity discount.


Consider the Freddie Mac bonds in the table.

| BB Ticker | BBID    | CUSIP     | Quote | Issue | Maturity | Call | Days to Call | Issue Size | Cpn Freq | Cpn Rate | Clean Price | Dirty Price |
|----------|---------|-----------|------------|------------|---------|---------|--------|------------|---------------|------|-------|-------------|
| FHLMC 5.8 01/28/39 Corp    | COZF5837751 | 3134H1QQ9 | 2024-02-21 | 2024-01-28  | 2039-01-28 | 2024-07-28 | 158 | $15 million | Semi-Annual | 5.80%       | $98.890 | 99.244|
| FHLMC 6 02/21/34 Corp     | COZD1131782 | 3134H1TN3 | 2024-02-21 | 2024-02-21  | 2034-02-21 | 2024-08-21 | 182 | $15 million | Semi-Annual | 6.00%       | $100.000 | 100.000 |


### Callable Bond

One important difference between Treasury debt and this bond is that this bond is **callable**
* Note that the holder of the bond is short this call option; (the issuer has the optionality.) 

#### Style
* Bermudan style. That is, exercised on a set of (quarterly) dates.
* Here, we simplify to European style.
* This simplification would be fairly accurate for callable bonds where the first Bermudan date is several years out. 
* And many callable bonds are European exercise, but they tend to be corporates (often banks) and we want to avoid dealing with credit risk in this course.

#### Strike
* Strike is 100.
* The strike is based on the **clean** price of the bond, meaning the price which does not account for accrued interest.
* For this bond, the call dates are coupon dates, so the clean and dirty prices are the same.

### Forward Bond Price

For Black's formula, we need the **forward** bond price. 

This is straightforward to calculate, though it requires a few assumptions. 

$$P_{\text{forward}}(T_\text{option}\to T) = \frac{P(T) - \sum_{i=1}^n Z(T_i)C_i}{100Z(T_\text{option})}$$

where $n$ denotes the number of cashflows (coupons) between $t$ and $T_{\text{option}}$, and $C_i$ denote the size of those coupons.

### Implied Volatility

What is the implied vol of the bond price?
* Suppose you know that the implied vol of rates is `2.68%`.
* We know that up to a linear approximation, the vol of rates will scale to the vol of bonds via the duration of the bond.

Thus, we can approximate...

$$\sigma_{\text{bond}} \approx D \times \sigma_{\text{rate}}\times r$$

***

## Pricing

#### Approach
The approach is to decompose the Freddie Mac callable bond into...

$$\text{callable bond} = \text{vanilla bond} - \text{call option on vanilla bond}$$

Price the vanilla bond
* Use the basic bond pricing formula to price each of the two vanilla bonds.

Price the embedded option
* Use Black's formula to price this implicit call option and the resulting callable bond.

Price the callable bond
* Note that the price of the callable bond is the value of the vanilla bond minus this American option.

How close is the modeled price to the market quoted price now that you are accounting for the short embedded call option? 

### Solving for Implied Volatility

What is the market's implied value of the embedded option?

Use this to solve for the implied price volatility and implied rate volatility.

### OAS

The **option adjusted spread** is the spread one would need to add to the spot curve, (or the forward curve), to get the modeled price to match the market price of the derivative. 

That is, how many bps would you need to add or subtract to all the spot rates in order to perfectly price this callable bond? 

* Ignore the effect of the rate curve shift on implied volatilities.

* Ignore the effect of the rate curve shift on cashflows.

This **OAS** is a common way to quote whether a derivative seems to be undervalued (positive OAS) or overvalued (negative OAS.) Of course, the OAS is dependent on our modeling choices, so it could also indicate the model is bad.

***

## Differences with Callable Bonds

Note a few things
* there is negative convexity in the callable bond
* callable bond's price is lower at all rate levels
* callable bond's value does not go above par for any rate level

<img src="../refs/negative_convexity.png" width="1000">

.

<img src="../refs/callable_vs_vanilla.png" width="1000">

***

## Exercising the Option

Re-value each bond at their respective call dates.

Should Freddie Mac call either bond?

### Black's Assumptions

Which assumptions of Black's formula do we prefer to Black-Scholes for this problem?

### Are Callable Bonds Overpriced?

Our modeled price of the callable bond doesn't exactly match the market's quoted price.

Our model could be poorly...
* **calibrated** (settlement timing, discount rates, forward rates, implied vol)
* **modeled** (market frictions, non-Brownian dynamics, etc.)

But the market could be mispricing these bonds!
* Callable bonds are seemingly overpriced in many situations.
* Perhaps the issuer does not exercise (call) at optimal times.
* Why might this overpricing be hard to arbitrage?
* Francis Longstaff (1992) has a good discussion of mispriced callable Treasury bonds.

***

# **<span style="color:red">Solution 2</span>**

### Parameters of the Bond

In [3]:
import numpy as np
import pandas as pd

import sys
sys.path.insert(0, '../cmds')
from ficcvol import *
#from binomial import *
from ratecurves import *

import datetime
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)

In [4]:
DATE = '2024-02-21'
curves = pd.read_excel(f'../data/cap_curves_{DATE}.xlsx', sheet_name=f'rate curves {DATE}').set_index('tenor')

In [5]:
curves

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052237,0.052237,0.987109,NaN,NaN,NaN
0.50,0.051662,0.051658,0.974663,0.051080,0.165977,0.165977
0.75,0.050705,0.050690,0.962926,0.048756,0.191234,0.210421
1.00,0.049585,0.049554,0.951944,0.046145,0.216491,0.253934
1.25,0.047977,0.047913,0.942202,0.041357,0.257488,0.352958
1.50,0.046607,0.046515,0.932981,0.039534,0.288170,0.371568
1.75,0.045506,0.045391,0.924052,0.038654,0.310071,0.378711
2.00,0.044605,0.044471,0.915348,0.038032,0.324724,0.376913
2.25,0.043634,0.043474,0.907295,0.035506,0.333665,0.366914


In [6]:
CALC_FWD = True
CALC_VOL = True

T=10
FACE=100
cpn_freq = 2
cpn = .06

In [7]:
Topt = 0.5
STRIKE = 100
CLEANCALL = True

In [8]:
from bondmath import bond_pricer_formula, bond_pricer_dcf

px_vanilla_alt = bond_pricer_formula(T,curves.loc[T,'swap rates'],cpn=cpn,freq=cpn_freq)

temp = curves[['discounts']].copy().iloc[1::2]
temp.loc[T:,'cashflow'] = 0
temp.loc[:T,'cashflow'] = FACE * cpn/cpn_freq
temp.loc[T,'cashflow'] += FACE

temp['pv cf'] = temp.prod(axis=1)
px_vanilla = temp['pv cf'].sum()

pd.DataFrame([px_vanilla, px_vanilla_alt],index=['spot rates', 'swap rate'],columns=['clean price']).style.format('${:.2f}')

,clean price
spot rates,$117.03
swap rate,$117.27


In [9]:
px_vanilla_Topt = bond_pricer_formula(Topt,curves.loc[Topt,'spot rates'],cpn=cpn,freq=cpn_freq)
if CALC_FWD:    
    px_fwd = FACE * px_vanilla / px_vanilla_Topt
else:
    px_fwd = 103.31

pd.DataFrame([px_vanilla,px_vanilla_Topt,px_fwd],index=[f'maturity {T:.1f}',f'maturity {Topt:.1f}',f'forward from {Topt:.1f} to {T:.1f}'],columns=['price']).style.format('${:.2f}')

,price
maturity 10.0,$117.03
maturity 0.5,$100.41
forward from 0.5 to 10.0,$116.56


In [10]:
if CALC_VOL:
    from bondmath import duration_closed_formula
    
    duration = duration_closed_formula(T,curves.loc[T,'spot rates'],cpn,freq=cpn_freq)
    vol_bond = duration * curves.loc[Topt,'flat vols'] * curves.loc[T,'forwards']
    
    pd.DataFrame([curves.loc[Topt,'flat vols'], duration, vol_bond],index=['rate vol (flat)','bond duration','bond vol'],columns=['estimates']).style.format('{:.2%}')
else:
    vol_bond = .0268

In [11]:
px_calloption = blacks_formula(Topt,vol_bond,STRIKE,px_fwd,discount=curves.loc[Topt,'discounts'])
px_callable_bond = px_vanilla - px_calloption

tab = pd.DataFrame([px_vanilla,px_calloption,px_callable_bond],index=['vanilla bond','call option','callable bond'],columns=['formulaic prices'])
tab.style.format('{:.2f}')

,formulaic prices
vanilla bond,117.03
call option,16.14
callable bond,100.89


***

In [12]:
T=9.5
Topt = 0.5

In [13]:
DATE = '2024-08-21'
curves = pd.read_excel(f'../data/cap_curves_{DATE}.xlsx', sheet_name=f'rate curves {DATE}').set_index('tenor')

In [14]:
#from bondmath import bond_pricer_formula, bond_pricer_dcf

px_vanilla_alt = bond_pricer_formula(T,curves.loc[T,'swap rates'],cpn=cpn,freq=cpn_freq)

temp = curves[['discounts']].copy().iloc[1::2]
temp.loc[T:,'cashflow'] = 0
temp.loc[:T,'cashflow'] = FACE * cpn/cpn_freq
temp.loc[T,'cashflow'] += FACE

temp['pv cf'] = temp.prod(axis=1)
px_vanilla = temp['pv cf'].sum()

pd.DataFrame([px_vanilla, px_vanilla_alt],index=['spot rates', 'swap rate'],columns=['clean price']).style.format('${:.2f}')

,clean price
spot rates,$121.53
swap rate,$121.69


In [15]:
px_vanilla_Topt = bond_pricer_formula(Topt,curves.loc[Topt,'spot rates'],cpn=cpn,freq=cpn_freq)
if CALC_FWD:    
    px_fwd = FACE * px_vanilla / px_vanilla_Topt
else:
    px_fwd = 103.31

pd.DataFrame([px_vanilla,px_vanilla_Topt,px_fwd],index=[f'maturity {T:.1f}',f'maturity {Topt:.1f}',f'forward from {Topt:.1f} to {T:.1f}'],columns=['price']).style.format('${:.2f}')

,price
maturity 9.5,$121.53
maturity 0.5,$100.65
forward from 0.5 to 9.5,$120.74


In [16]:
if CALC_VOL:
    from bondmath import duration_closed_formula
    
    duration = duration_closed_formula(T,curves.loc[T,'spot rates'],cpn,freq=cpn_freq)
    vol_bond = duration * curves.loc[Topt,'flat vols'] * curves.loc[T,'forwards']
    
    pd.DataFrame([curves.loc[Topt,'flat vols'], duration, vol_bond],index=['rate vol (flat)','bond duration','bond vol'],columns=['estimates']).style.format('{:.2%}')
else:
    vol_bond = .0268

In [17]:
px_calloption = blacks_formula(Topt,vol_bond,STRIKE,px_fwd,discount=curves.loc[Topt,'discounts'])
px_callable_bond = px_vanilla - px_calloption

tab = pd.DataFrame([px_vanilla,px_calloption,px_callable_bond],index=['vanilla bond','call option','callable bond'],columns=['formulaic prices'])
tab.style.format('{:.2f}')

,formulaic prices
vanilla bond,121.53
call option,20.26
callable bond,101.26


***